In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import baycomp_plotting as bplt
import baycomp as bc


from sslearn.wrapper import SelfTraining, Rasco, RelRasco, RotRelRasco
from sslearn.supervised.rotation import RotationForestClassifier, RotatedTree
from sslearn.model_selection import StratifiedKFoldSS
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

import os


In [2]:
path = "/home/jlgarridol/Dropbox/UBU/Research/SSRotation/csv"

seed = 100

classifiers = {
    "Rot-Rel-Rasco": RotRelRasco(base_estimator=DecisionTreeClassifier(random_state=seed), n_estimators=10, random_state=seed, incremental=False),
    "Rel-Rasco": RelRasco(base_estimator=DecisionTreeClassifier(random_state=seed), n_estimators=10, random_state=seed, incremental=False),
    "Rasco": Rasco(base_estimator=DecisionTreeClassifier(random_state=seed), n_estimators=10, random_state=seed, incremental=False),
    "SelfTraining": SelfTraining(base_estimator=RotationForestClassifier(base_estimator=DecisionTreeClassifier(random_state=10), n_estimators=10, random_state=seed), max_iter=40)
}


acc_trans = {
    "Rot-Rel-Rasco": [],
    "Rel-Rasco": [],
    "Rasco": [],
    "SelfTraining": []
}

acc_ind = {
    "Rot-Rel-Rasco": [],
    "Rel-Rasco": [],
    "Rasco": [],
    "SelfTraining": []
}


In [28]:
datasets = {}
for file in os.listdir(path):
    dataset = pd.read_csv(os.path.join(path,file), header=None)

    columns = []
    for i, tp in enumerate(dataset.dtypes):
        if not np.issubdtype(tp, np.number) and i != dataset.shape[1]-1:
            columns.append(i)
            dataset[i] = dataset[i].astype("|S")
    
    y = dataset.iloc[:,-1]
    if np.issubdtype(y, np.number):
        y = y+2 
    X = dataset.iloc[:,:-1]
    if len(columns) > 0:
        elements = [X[X.columns.difference(columns)]]

        for col in columns:
            elements.append(pd.get_dummies(X[col]))
        
        concatenated_data = pd.concat(elements, axis=1)
        X = concatenated_data

    
    
    datasets[file.split(".")[0]] = (X.to_numpy(), y.to_numpy())

In [45]:
skf = StratifiedKFoldSS(n_splits=10, random_state=seed, shuffle=True)

for d in datasets:
    X, y = datasets[d]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=float(3/4), random_state=seed)
    print(d)
    for learner in classifiers:
        print("\t",learner)
        st = classifiers[learner]
        scores = []
        scores_ind = []
        for X_, y_, _, unlabel in skf.split(X_train,y_train):
            st.fit(X_,y_)
            scores.append(st.score(X_train[unlabel], y_train[unlabel]))
            scores_ind.append(st.score(X_test, y_test))
        acc_trans[learner].append(np.mean(scores))
        acc_ind[learner].append(np.mean(scores_ind))

vowel
	 Rot-Rel-Rasco
	 Rel-Rasco
